In [ ]:
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay,
    f1_score, roc_curve, roc_auc_score
)
import pandas as pd
import joblib
import os
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

In [82]:
model = joblib.load("../Modeling/classifier.joblib")
model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=-1,
              num_parallel_tree=None, ...)

In [23]:
# 1. Load and concatenate all CSV files in a folder
def load_and_concatenate(folder_path):
    dataframes = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            dataframes.append(df)
    return pd.concat(dataframes, ignore_index=True)


def apply_smote(X_train, y_train):
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    return X_resampled, y_resampled

In [24]:
folder_path = 'D:/College/Third Year/Second Term/Pattern/Project/speaker-gender-age-recognition/features_200'
target = 'label'
data = load_and_concatenate(folder_path)
# data.to_csv('all_features.csv', index=False)
# df = pd.read_csv(folder_path)
# data = df

print('loaded')
print(data.shape)

data.dropna(inplace=True)
X = data.drop(columns=[target])
y = data[target]


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_resampled, y_train_resampled = apply_smote(X_train_scaled, y_train)
print('smoted')

loaded
(152134, 202)
smoted


In [39]:

def evaluate_model(y_true, y_pred, y_pred_proba=None, class_names=None):
    # Basic metrics
    acc = accuracy_score(y_true, y_pred)
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    weighted_f1 = f1_score(y_true, y_pred, average='weighted')
    report = classification_report(y_true, y_pred, target_names=class_names)
    
    print(f"Accuracy: {acc:.4f}")
    print(f"Macro F1-score: {macro_f1:.4f}")
    print(f"Weighted F1-score: {weighted_f1:.4f}")
    print("\nClassification Report:\n", report)

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    annotations = [[str(cell) for cell in row] for row in cm]
    fig_cm = go.Figure(data=go.Heatmap(
        z=cm,
        x=class_names,
        y=class_names,
        colorscale='Blues',
        showscale=True,
        text=annotations,
        texttemplate="%{text}",
        hoverinfo="z"
    ))
    fig_cm.update_layout(
        title="Confusion Matrix",
        xaxis_title="Predicted Label",
        yaxis_title="True Label"
    )
    fig_cm.show()


    # ROC Curve (multi-class)
    if y_pred_proba is not None:
        y_true_bin = label_binarize(y_true, classes=list(range(len(class_names))))
        fig_roc = go.Figure()
        for i in range(len(class_names)):
            fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
            fig_roc.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC - {class_names[i]}'))
        fig_roc.add_trace(go.Scatter(x=[0,1], y=[0,1], mode='lines', name='Random', line=dict(dash='dash')))
        fig_roc.update_layout(
            title="ROC Curves",
            xaxis_title="False Positive Rate",
            yaxis_title="True Positive Rate",
            showlegend=True
        )
        fig_roc.show()

        auc_score = roc_auc_score(y_true_bin, y_pred_proba, average='macro')
        print(f"Macro ROC AUC Score: {auc_score:.4f}")


In [ ]:
y_pred = model.predict(X_test_scaled)
y_true = y_test
y_pred_proba = model.predict_proba(X_test_scaled)

In [74]:
evaluate_model(y_true, y_pred, y_pred_proba=y_pred_proba, class_names=["M20", "F20", "M50", "F50"])

Accuracy: 0.8776
Macro F1-score: 0.8155
Weighted F1-score: 0.8816

Classification Report:
               precision    recall  f1-score   support

         M20       0.95      0.90      0.92     21670
         F20       0.85      0.89      0.87      3235
         M50       0.56      0.70      0.62      3008
         F50       0.83      0.87      0.85      2514

    accuracy                           0.88     30427
   macro avg       0.80      0.84      0.82     30427
weighted avg       0.89      0.88      0.88     30427



Macro ROC AUC Score: 0.9653


In [ ]:
y_pred = model.predict(X_train_resampled)
y_true = y_train_resampled
y_pred_proba = model.predict_proba(X_train_resampled)

In [ ]:
evaluate_model(y_true, y_pred, y_pred_proba=y_pred_proba, class_names=["M20", "F20", "M50", "F50"])

Accuracy: 0.9494
Macro F1-score: 0.9494
Weighted F1-score: 0.9494

Classification Report:
               precision    recall  f1-score   support

         M20       0.93      0.92      0.93     86679
         F20       0.96      0.97      0.97     86679
         M50       0.94      0.94      0.94     86679
         F50       0.97      0.96      0.97     86679

    accuracy                           0.95    346716
   macro avg       0.95      0.95      0.95    346716
weighted avg       0.95      0.95      0.95    346716



Macro ROC AUC Score: 0.9959


In [ ]:
from xgboost import XGBClassifier


model = XGBClassifier(n_estimators=200,
        eval_metric='mlogloss',
        random_state=42)

model.fit(
    X_train_resampled,
    y_train_resampled,
    verbose=True,
)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define base models
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('xgb', XGBClassifier(n_estimators=100, learning_rate=0.05, random_state=42, eval_metric='mlogloss') )
]

# Meta-model
meta_model = LogisticRegression(max_iter=1000)

# Create stacking ensemble
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    n_jobs=-1,
    verbose=1
)


In [81]:
# Train stacking model
stacking_model.fit(X_train_resampled, y_train_resampled)

LightGBMError: bad allocation

In [ ]:
# Optional: Evaluate
y_pred = stacking_model.predict(X_test_scaled)
print("Stacking Accuracy:", classification_report(y_test, y_pred))